In [1]:
import pandas as pd
import numpy as np

# Imports of custom moduls
import Multitours as MT
import BnP
import instance
from solution2 import importData2, importData3

# Data

In [55]:
# Read excel file for rows 1 to 9 for demanda data
ncustomers = 8
nvehicles = 4
vehicleCapacity = [60000 for i in range(nvehicles)]
Customers = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example1.xlsx', sheet_name='Sheet1', nrows=ncustomers, usecols="A:E")
Time = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example1.xlsx', sheet_name='Sheet1', skiprows=ncustomers+2, usecols="A:C")
Vehicles = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example1.xlsx', sheet_name='Sheet1', nrows=nvehicles, usecols="G:I")
delta = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example1.xlsx', sheet_name='Sheet1', nrows=1, usecols="K:K")
Network = Time[["Origin", "Destination"]]
Customers["Tmax"] = Customers["Capacity"] / Customers["Demand"]

### Parameters

In [3]:
# Vehicle cost parameter
psi = [1000 for i in range(nvehicles)]

# Transportation cost
delta = 100

# Delivery cost
# column Phi of df Costumers with indeces 1 to ncustomers
phi = Customers["Phi"].copy()
phi.index = phi.index + 1

# Holding Cost
h = Customers["Holding"].copy()
h.index = h.index + 1

# Demand
d = Customers["Demand"].copy()
d.index = d.index + 1

# Capacity
k = Customers["Capacity"].copy()
k.index = k.index + 1

# Travel time parameter
shape = (ncustomers + 1, ncustomers + 1)
t = np.full(shape, 100)

for index, row in Time.iterrows():
    i = row['Origin']
    j = row['Destination']
    t[i, j] = row['Time']

# Create a dictionary with all the parameters
parameters = {
    "ncustomers": ncustomers,
    "nvehicles": nvehicles,
    "vehicleCapacity": vehicleCapacity,
    "psi": psi,
    "delta": delta,
    "phi": phi,
    "h": h,
    "d": d,
    "k": k,
    "t": t
}


## Initial set of Multitours and Cycle time

### Random and Basic tours

In [22]:
MultiTours,Cost,T,X = instance.Initial_Tours(Random = True, Basic1 = True, n_radom_multitours=20, Network = Network, Time = Time, Customers = Customers, instance_parameters = parameters)

### Big M (Artificial Variable)

In [24]:
MultiTours,Cost,T,X = instance.Artificial_Variable(ncustomers, BigM=1000000)

### Stored Solutions

In [6]:
# Stored solution from example 2 (CG heuristic)
# MultiTours,Cost,T,X = importData2(ncustomers)
# MultiTours,Cost,T,X = importData3(ncustomers)

# Branch and Price

In [7]:
# Min_TC, solutionMP, tree = BnP.BnP(MultiTours,Cost,T,X, parameters, strategy="MultiTours")
Min_TC, solutionMP, tree = BnP.BnP(MultiTours,Cost,T,X, parameters, strategy="Trips")

   Depth       TC   Leaf            UB       LB       Gap
0      0  87300.0  False  89806.666667  87300.0  2.871325
1      1  87300.0   True  87300.000000  87300.0  0.000000
2      1  88557.5   True  87300.000000  87300.0  0.000000

Optimal solution found
Min total cost: 87300.0
Solution: [(8, 1.0), (20, 1.0), (21, 1.0), (26, 1.0)]
[0, 8, 0] 2.0 11200.0
[0, 5, 6, 0] 3.0 23700.0
[0, 1, 2, 7, 0] 4.0 31250.0
[0, 4, 3, 0] 4.0 21150.0


In [8]:
# 16 minutes to run BnP with Multitours strategy (saved root node solution from example 3)